In [125]:
import pandas as pd
import re
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [64]:
df = pd.read_csv('diffusion_db_unaltered.csv')

In [22]:
df.head()

,prompt,image_nsfw,prompt_nsfw
0,"a portrait of a female robot made from code, v...",0.554853,0.001621
1,a portrait of a female robot made from a cloud...,0.153645,0.000707
2,"only memories remain, trending on artstation",0.062496,0.000425
3,dream swimming pool with nobody,0.030799,0.003586
4,a dog doing weights. epic oil painting.,0.181035,0.030822


In [67]:
violent_keywords = [
    "killing", "attacks", "attacking", "gun", "shoot", "stab", "murder", 
    "bomb", "threat", "harm", "suicide", 'beat', 
    'choke', 'hit', 'punch', 'slaughter', 'strangle', 
    'suffocate', 'crush', 'assault', 'batter', 'maul', 
    'knife', 'rifle', 'pistol', 'blade', 'explosive', 
    'grenade', 'missile', 'poison', 'chainsaw', 'crowbar', 
    'bat', 'taser', 'detonate', 'slash', 'eliminate',
] # REMEMBER TO UPDATE LATER AS I FIGURE OUT WHAT WORDS SHOW UP OFTEN :nooticing:
# update 1: my algorithm is slow asl because i have like 50 or smth words, reducing word count might actually be the method.

In [56]:
pattern = r"\b(" + "|".join([f"{word}" for word in violent_keywords]) + r")\b" #regex patterns, can be changed depending on if i need case sensitivity or not
regex = re.compile(pattern, flags=re.IGNORECASE) 

In [7]:
size = 100000  # can be adjusted, may want to increase scale later depending on how this performs. UPDATE 1: performance is based on violent_keywords, might need to adjust that list to improve performance
output_list = []
for chunk in pd.read_csv('diffusion_db_unaltered.csv', chunksize=size):
    chunk['contains_violent'] = chunk['prompt'].str.contains(
        pattern, case=False, regex=True, na=False
    ) # try to improve performance, currently takes 5~ minutes to process 100k, haven't tested 200k+

    # VERSION 1 OF THIS DONT RUN IT THIS IS HERE FOR REFERENCE

In [65]:
#attempt 2 at chunking
chunk_size = 100000 # 200k is probably the limit because of memory/speed issues, needs to be tested further
output_list = []
for chunk in pd.read_csv('diffusion_db_unaltered.csv', chunksize=chunk_size):
    chunk['contains_violent'] = chunk['prompt'].str.contains(
        regex,
        na=False # basic substring check, might need to backtrack to v1 substring check
)
violent_rows = chunk[chunk['contains_violent']] #specifically only saving flagged rows to try and optimize, might change later
output_list.append(violent_rows)
if output_list: # combining results, may skip this step later
    final_df = pd.concat(output_list, ignore_index=True)
else:
    print("yea there aint no violence here gang")

C:\Users\dngo6\AppData\Local\Temp\ipykernel_34364\1945423044.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk['contains_violent'] = chunk['prompt'].str.contains(
C:\Users\dngo6\AppData\Local\Temp\ipykernel_34364\1945423044.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk['contains_violent'] = chunk['prompt'].str.contains(
C:\Users\dngo6\AppData\Local\Temp\ipykernel_34364\1945423044.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk['contains_violent'] = chunk['prompt'].str.contains(
C:\Users\dngo6\AppData\Local\Temp\ipykernel_34364\1945423044.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk['contains_vi

In [66]:
testing = final_df.sample(10)
print(testing[["prompt", "contains_violent"]])

                                                                                                                                                                                                                                                                                                                                                                                                                  prompt  contains_violent
434                                                                                                                                                                                                                                                                                                                                                      die cut sticker, goku, gatling attack by luffy, splatter paint               True
243                                                                                                                                       highly d

In [114]:
sampled_data = pd.read_csv('diffusion_db_unaltered.csv', nrows=2000000)
random_sample = sampled_data.sample(n=200000, random_state=5)  # Randomly sample 500,000 rows
chunk_size = 200000
output_list = []
for start in range(0, len(random_sample), chunk_size):
    # Take a chunk from the random sample
    chunk = random_sample.iloc[start:start + chunk_size]
    # Check for explicit content using the regular expression
    chunk['contains_violent'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)
    chunk['contains_explicit'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)
    # Filter for rows with explicit content
    output_list.append(chunk)
if output_list:
    final_df = pd.concat(output_list, ignore_index=True)
else:
    print("No violent content found.")

C:\Users\dngo6\AppData\Local\Temp\ipykernel_34364\1867132658.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk['contains_violent'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)
C:\Users\dngo6\AppData\Local\Temp\ipykernel_34364\1867132658.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['contains_violent'] = chunk['prompt'].str.contains(pattern, case=False, regex=True, na=False)
C:\Users\dngo6\AppData\Local\Temp\ipykernel_34364\1867132658.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  chunk['contains_explicit'] = chunk['pr

In [71]:
final_df

,prompt,image_nsfw,prompt_nsfw,contains_violent
0,"portrait, 3 d render, tall, sexy, little fat, anthropomorphic female deer, wearing along white dress, in the style of zootopia,",0.319462,0.339705,False
1,"a dusty attic, servants quarters, two small iron beds, 1800s historic photographs",0.045036,0.005156,False
2,"giant white daisy flower crown over the head, girl standing on cliff, surreal photography, solar eclipse, milky way, dramatic light, impressionist painting, clouds, digital painting, artstation, james gilleard, liam wong, jeremy mann, simon stalenhag",0.073490,0.009478,False
3,deep chaos symbolism 4 7 rubies 3 diamonds crown of thorns,2.000000,0.013615,False
4,"brazilian favela, national geographic, canon 5 0 mm, cinematic lighting, photography, retro, film, kodachrome",0.117835,0.001135,False
...,...,...,...,...
199995,ink art by xu wei - c 2 0,0.150621,0.001292,False
199996,a painting in the style of william henry millais and in the style of gil elvgren.,0.936622,0.000362,False
199997,"a of a face transplant in an operating room with robotic surgeons , photorealistic,by Wlop,4k resolution",0.125575,0.012488,False
199998,"i will stay with you as the universe ends, intricate artwork by tooth wu and wlop and beeple. octane render, trending on artstation, greg rutkowski very coherent symmetrical artwork. cinematic, hyper realism, high detail, octane render",0.126242,0.000670,False


In [59]:
final_df.loc[11630]

prompt              Danny DeVito as a titan in attack on titan
image_nsfw                                            0.184595
prompt_nsfw                                           0.007746
contains_violent                                          True
Name: 11630, dtype: object

In [115]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)


In [63]:
dfff = final_df.replace({True: 1, False: 0})

In [168]:
violent_rows = chunk[chunk['contains_violent']]
violent_rows = violent_rows.replace({True: 1, False: 0})
non_violent_rows = chunk[~chunk['contains_violent']]
non_violent_rows = non_violent_rows.replace({True: 1, False: 0})

In [87]:
violent_rows

,prompt,image_nsfw,prompt_nsfw,contains_violent
1105087,cocnert for cello and atomic bomb,0.051783,0.066775,1
1099840,Kirby holding gun.,0.126990,0.045630,1
491929,tilt shift image of retro mid century office building parking lot in sunshine giant squid attack,0.074271,0.001137,1
1310720,"35mm color, lost photo, portrait, fashion shoot, weird, random, strange, spooky, hyperdetailed, photorealistic, high fashion, interesting, morgue, by David Cronenberg and WETA digital, !!!Vogue magazine!!! halo. octane rendering, cinematic, hyperrealism, octane rendering, 8k, depth of field, bokeh. iridescent accents. vibrant.",0.208033,0.001933,1
1972836,"vogue photo shoot, chic, feminine, 8 0 s skirt suit, pantyhose, stilettos, photorealistic, highly detailed, 4 k, by yasuhiro wakabayashi",0.484698,0.002063,1
...,...,...,...,...
601792,"a beefy man with a machine gun, egyptian pyramid in the background, 4 k, sharp focus, real life photo, highly detailed, cinematic, photorealistic, cyberpunk, book cover",0.826466,0.023059,1
156001,"« vanu soldier, full helmet, purple and black clothes, tall young male, dominant, heavy gun »",0.122416,0.033809,1
1430072,"wooden houses in hiroshima, atom bomb cloud in distance, colored pencil art",0.103364,0.005577,1
686750,"painting by greg rutkowski, dragons attack, epic, middle ages, fantasy, castle, matte painting illustration hearthstone, by greg rutkowski by greg tocchini by james gilleard",0.117775,0.002427,1


In [99]:
non_violent_rows["prompt_nsfw"].mean()

0.024992168707536688

In [170]:
final_df.loc[final_df['contains_violent'] == 1, "contains_violent"] = "Harmful"
final_df.loc[final_df['contains_violent'] == 0, "contains_violent"] = "Harmless"

In [171]:
final_df

,prompt,image_nsfw,prompt_nsfw,contains_violent,contains_explicit
0,"Emma watson Muscular and powerful medieval noble woman portrait, sci-fi, amber eyes, face, long hair, fantasy, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus,8k, illustration, art by artgerm and greg rutkowski and alphonse mucha",0.723663,0.001068,Harmless,False
1,"epic professional digital image of a hybrid girl of plants and flowers, fox digs, reina rachin, ignacio fernandez rios, leesha hannigan, wayne haag, artstation, cgsocietywlop, epic, a lot of wow, a lot of detail, gorgeous, detailed, cinematic, masterpiece",0.487867,0.002508,Harmless,False
2,scene from mupet save private ryan,0.050685,0.002552,Harmless,False
3,"anime, mage, vogue, intricate oil painting, trending on artstation, john singer sargent, greg rutkowski, side profile, pink hair, masquerade",0.283684,0.001868,Harmless,False
4,"humanoid rat who has bald head, weak chin clean shaven and is insecure",0.053884,0.950304,Harmless,False
...,...,...,...,...,...
199995,"cnn breaking news, mar a lago has been violently occupied by tony montana from the film scarface ( 1 9 8 3 )",0.039752,0.000935,Harmless,False
199996,"A robot talking to Gandalf the Grey in a beach scene, Cinematic shot, Highly Detailed",0.079238,0.000419,Harmless,False
199997,A fairy witch eating mushrooms in a magical forest,0.735223,0.327841,Harmless,False
199998,"conversion van interior, van life",0.093359,0.001529,Harmless,False


In [172]:
neigh = KNeighborsClassifier(n_neighbors=5)
clf = MultinomialNB()

In [173]:
y = (final_df['prompt_nsfw']>=0.24992168707536688)

In [174]:
X = final_df[['contains_violent']]

In [175]:
neigh.fit(X, y)

ValueError: could not convert string to float: 'Harmless'

In [144]:
clf.fit(X, y)

MultinomialNB()

In [159]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [160]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

In [161]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [162]:
y_pred = knn.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [163]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.97275

In [164]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred, average='macro')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.486375

In [165]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred, average='macro')

0.5

In [166]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='macro')

0.493093397541503

In [167]:
y_pred_bayes = clf.predict(X)

NotFittedError: This MultinomialNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [149]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred_bayes)

0.97297

In [150]:
from sklearn.metrics import precision_score
precision_score(y, y_pred_bayes, average='macro')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.486485

In [151]:
from sklearn.metrics import recall_score
recall_score(y, y_pred_bayes, average='macro')

0.5

In [152]:
from sklearn.metrics import f1_score
f1_score(y, y_pred_bayes, average='macro')

0.4931499211848127

In [176]:
y_pred_bayes

array([False, False, False, ..., False, False, False])